# Calculating the standar deviation for MR sim </br>
Using T1 and T2* or R1 and R2* maps to calculate the standar deviation to be used during simulations in the SC wm and SC gm. </br>
Following results from Smith et al. 2008, we can relate the deep gray matter (DGM) regions of the brain to the spinal cords gray matter and we can use the dense fiber regions of the brain to the spinal cords white matter.

In [1]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### First, using the R1 and R2* maps from the QSM RC2 challenge </br>
Disclaimer: It is 7T data

In [5]:
# Load niftis
qsm_rc2_r2s_img = nib.load(r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/R2star.nii.gz")
qsm_rc2_r2s_data = qsm_rc2_r2s_img.get_fdata()
qsm_rc2_r1_img = nib.load(r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/R1.nii.gz")
qsm_rc2_r1_data = qsm_rc2_r1_img.get_fdata()
# We also want to load the segmentation file 
qsm_rc2_segs_data = nib.load(r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/SegmentedModel.nii.gz").get_fdata()
# We also need a corpus callosum mask
qsm_rc2_corpus_callosum_msk_data = nib.load(r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/Corpus_callosum.nii.gz").get_fdata()


The segmentantion model list as follows:
1. Caudate Nucleus (DGM)
2. Globalus Palladis (DGM)
3. Putamen (DGM)
4. Red Nucleus (DGM)
5. Dentate Nucleus (DGM)
6. Substantia nigra (DGM)
7. Thalamus (DGM) </br>

The other masks on the model are: </br>
White matter, Gray matter, CSF, blood, fat, bone, air, muscle and calcification

In [6]:
# We need to invert the R2star and R1 maps to get the T2star and T1 maps
qsm_rc2_t2s_data = 1 / qsm_rc2_r2s_data
qsm_rc2_t1_data = 1 / qsm_rc2_r1_data

# Now we can save this new data as nifti files
qsm_rc2_t2s_img = nib.Nifti1Image(qsm_rc2_t2s_data, qsm_rc2_r2s_img.affine)
nib.save(qsm_rc2_t2s_img, r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/T2star.nii.gz")
qsm_rc2_t1_img = nib.Nifti1Image(qsm_rc2_t1_data, qsm_rc2_r1_img.affine)
nib.save(qsm_rc2_t1_img, r"E:\msc_data\sc_qsm\qsm_RC2_phantom_data/T1.nii.gz")

Loading the respective masks on the inverted R2* [Hz] map: </br>
I. For SC white matter, on our 3 regiions for corpus callosum we have: 0.02, 0.0052 and 0.0122 => 0.0124667 [s] => 12.4 [ms] </br>
II. For SC gray matter, on our 7 regions of DGM we have: 0.0198, 0.0093, 0.0064, 0.0032, 0.0042, 0.0051, 0.0203 => 0.0097571 [s] => 9.757[ms] </br>

Loading the respective masks on the inverted R1 map: </br>
I. For SC white matter, on our 3 regions for corpus callosum we have: 0.1628, 0.1292, 0.1327 => 0.1415 [s] ==> 141.5 [ms]</br>
II. For SC gray matter on our 7 regions of DGM we have: 0.2609, 0.636, 0.180, 0.129, 0.137, 0.215, 0.306 => 0.2662 [s] = 266.2 [ms] </br>

On the paper: https://pmc.ncbi.nlm.nih.gov/articles/PMC3508464
On 8 subjects, T2* at 3T of only the white matter regions classified as dense fiber we have that T2* std is: 4ms in GCC and 5.375ms for SPL. </br>
Average of T2* for WM: 4.6875 </br>
GCC is genu of corpus callosum 
SPL is splenium of carpus callosum


From the following data set: https://openneuro.org/datasets/ds004611/versions/1.0.2, we get T1 map @ 3T of subject MKP611 (17yo) and use SAMSEG to segment different regions of the brain. </br>
I. For Deep gray matter regions:
- Label 49: 134.5
- Label 10: 141.09
- Label 12: 89.003
- Label 51: 96.67
- Label 50: 116.27
- Label 11: 111.84 </br>
Average to: 114.895

II. For White matter we select the corpus callosum:
We have 2 sides, Label 41 and Label 2 with std of 105.72 and 106.58 respectively. </br>
Average to: 106.15


From the inverted relaxation maps acquired at 7 Tesla we see that at 3T, the T1 average decreases slightly for WM (141.5 -> 106.15) </br> 
But more prominently for GM going from 266.2 -> 114.895. </br>
